<a href="https://colab.research.google.com/github/VadimKhurs/BadComments/blob/main/bad_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt_tab')
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_curve
import numpy as np
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
df = pd.read_csv("data/labeled.csv")
df.shape

(14412, 2)

In [98]:
df["toxic"].value_counts()

,count
toxic,
0.0,9586
1.0,4826


In [99]:
df["toxic"] = df['toxic'].apply(int)

In [100]:
df["toxic"].value_counts()

,count
toxic,
0,9586
1,4826


In [101]:
for c in df[df["toxic"] == 1]["comment"].head(5):
  print(c)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



In [102]:
for c in df[df["toxic"] == 0]["comment"].head(5):
  print(c)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

In [103]:
train_df, test_df = train_test_split(df, test_size=500)

test_df.shape

(500, 2)

In [104]:
test_df["toxic"].value_counts()

,count
toxic,
0,315
1,185


In [105]:
sentence_example = df.iloc[1]["comment"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language='russian')
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

tokens

['Хохлы',
 ',',
 'это',
 'отдушина',
 'затюканого',
 'россиянина',
 ',',
 'мол',
 ',',
 'вон',
 ',',
 'а',
 'у',
 'хохлов',
 'еще',
 'хуже',
 '.',
 'Если',
 'бы',
 'хохлов',
 'не',
 'было',
 ',',
 'кисель',
 'их',
 'бы',
 'придумал',
 '.']

In [106]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
  tokens = word_tokenize(sentence, language="russian")
  tokens = [i for i in tokens if i not in string.punctuation]
  if remove_stop_words:
    tokens = [i for i in tokens if i not in russian_stop_words]
  tokens = [snowball.stem(i) for i in tokens]
  return tokens

tokenize_sentence(sentence_example)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [107]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))


In [108]:
features = vectorizer.fit_transform(train_df["comment"])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [109]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df["toxic"])

LogisticRegression(random_state=0)

In [118]:
pred_train = model.predict(features)

In [122]:
for i in np.where(pred_train == 0):
  print(i)
  print( train_df["comment"].iloc[i] )

[    4     6     7 ... 13905 13907 13910]
4038     Сейчас без камеры даже гриб-мухомор не сорвёшь...
8987         А как же радиоуправляемые машинки в клубах?\n
11374    Я понимаю) Это был крик души, за 6 месяцев в И...
11747    Плохо что тебя не интересуют особи женского по...
4794     Действительно век гаджетов, сейчас всё на бтар...
                               ...                        
7029     а движку на это пофигу? а то не хотелось бы на...
5516     Места замечательные, но лучше было когда никак...
5054     А сертификацией кто будет заниматься? придется...
11026    Вообще вполне логично, они не смогли её найти ...
7655     Тоже самое, даже в туалет лень идти, не говоря...
Name: comment, Length: 9999, dtype: object


In [125]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True)) ),
    ("model", LogisticRegression(random_state=0))
])

In [124]:
model_pipeline.fit(train_df["comment"], train_df["toxic"])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7c4bf493ee80>)),
                ('model', LogisticRegression(random_state=0))])